<a href="https://colab.research.google.com/github/david3080/llmapi/blob/main/3_reactagent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ReActエージェントを使ったRAG

1. LlamaIndex関連ライブラリのインストール
2. LLMのAPIキーをセットして、使用するLLMをセット
3. Wikipediaから米国都市のドキュメントをダウンロード
4. 各都市向けにReActエージェントを作成
5. ReActエージェントにIndexnodeを定義
6. エージェントを選択するためのトップレベルレトリバーを定義
7. クエリのテスト

### 1. LlamaIndex関連ライブラリのインストール

AuthropicのClaude3とOpenAIのGPT-4、GoogleのGemini、GroqのLlama3をLLMモデルとして利用し、Embedding用モデルはHuggingFaceのモデルを利用する。llama-indexは破壊的変更があったりするので、最新の[0.10](https://note.com/npaka/n/nb8acc1f63312)を指定しています。

In [ ]:
%pip install llama-index==0.10.33 llama-index-llms-anthropic llama-index-llms-openai llama-index-llms-gemini llama-index-llms-groq llama-index-embeddings-huggingface python-dotenv

### 2. LLMのAPIキーをセットして、使用するLLMをセット

In [ ]:
import os, logging, sys
from dotenv import load_dotenv
if load_dotenv() == False:
    from google.colab import userdata
    os.environ["ANTHROPIC_API_KEY"] = userdata.get("ANTHROPIC_API_KEY")
    os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
    os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
    os.environ["GROQ_API_KEY"] = userdata.get("GROQ_API_KEY")
from IPython.display import display, HTML

# ログレベルの設定
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG, force=True)

# Anthropicの設定
from llama_index.llms.anthropic import Anthropic
llm_anthropic = Anthropic(model="claude-3-opus-20240229")

# OpenAIの設定
from llama_index.llms.openai import OpenAI
llm_openai = OpenAI(model="gpt-4")

# Geminiの設定
from llama_index.llms.gemini import Gemini
llm_gemini =  Gemini(model="models/gemini-pro")

# Groqの設定
from llama_index.llms.groq import Groq
llm_groq = Groq(model="llama3-70b-8192")

# Embeddingモデルの設定
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
embed_model = HuggingFaceEmbedding(model_name="intfloat/multilingual-e5-small")

# Llamaの設定
from llama_index.core import Settings
Settings.embed_model = embed_model
Settings.chunk_size = 512
Settings.llm = llm_anthropic # 利用するLLMを指定

### 3. Wikipediaから米国都市のドキュメントをダウンロード

「トロント」「シアトル」「シカゴ」「ボストン」「ヒューストン」のWikipedia記事をダウンロードしてRAGパイプラインを作成します。

In [ ]:
wiki_titles = ["トロント", "シアトル", "シカゴ", "ボストン", "ヒューストン"]

from pathlib import Path
import requests

for title in wiki_titles:
    response = requests.get(
        "https://ja.wikipedia.org/w/api.php",
        params={
            "action": "query",
            "format": "json",
            "titles": title,
            "prop": "extracts",
            "explaintext": True,
        },
    ).json()
    page = next(iter(response["query"]["pages"].values()))
    wiki_text = page["extract"]

    data_path = Path("data/2_ReactAgent")
    if not data_path.exists():
        Path.mkdir(data_path)

    with open(data_path / f"{title}.txt", "w") as fp:
        fp.write(wiki_text)

# Load all wiki documents
from llama_index.core import SimpleDirectoryReader

city_docs = {}
for wiki_title in wiki_titles:
    city_docs[wiki_title] = SimpleDirectoryReader(
        input_files=[f"data/2_ReactAgent/{wiki_title}.txt"]
    ).load_data()

### 4. 各都市向けにReActエージェントを作成

In [ ]:
from llama_index.core.agent import ReActAgent
from llama_index.core import VectorStoreIndex, SummaryIndex
from llama_index.core.tools import QueryEngineTool, ToolMetadata

# Build agents dictionary
agents = {}

for wiki_title in wiki_titles:
    # build vector index
    vector_index = VectorStoreIndex.from_documents(
        city_docs[wiki_title],
    )
    # build summary index
    summary_index = SummaryIndex.from_documents(
        city_docs[wiki_title],
    )
    # define query engines
    vector_query_engine = vector_index.as_query_engine()
    summary_query_engine = summary_index.as_query_engine()

    # define tools
    query_engine_tools = [
        QueryEngineTool(
            query_engine=vector_query_engine,
            metadata=ToolMetadata(
                name="vector_tool",
                description=(
                    f"{wiki_title}から特定のコンテキストを取得するのに利用します."
                ),
            ),
        ),
        QueryEngineTool(
            query_engine=summary_query_engine,
            metadata=ToolMetadata(
                name="summary_tool",
                description=(
                    f"{wiki_title}に関連する回答をまとめるのに利用します."
                ),
            ),
        ),
    ]

    # build agent
    agent = ReActAgent.from_tools(
        query_engine_tools,
        llm=Settings.llm,
        verbose=True,
    )

    agents[wiki_title] = agent

### 5. ReActエージェントにIndexnodeを定義

In [12]:
from llama_index.core.schema import IndexNode

# define top-level nodes
objects = []
for wiki_title in wiki_titles:
    # define index node that links to these agents
    wiki_summary = (
        f"このコンテンツには{wiki_title}に関するウィキペディアの記事が含まれています."
        f"{wiki_title}に関する具体的な事実を調べる必要がある場合は、このインデックスを使用してください.\n"
        "複数の都市を分析する場合は、このインデックスを使用しないでください."
    )
    node = IndexNode(
        text=wiki_summary, index_id=wiki_title, obj=agents[wiki_title]
    )
    objects.append(node)

### 6. エージェントを選択するためのトップレベルレトリバーを定義

In [ ]:
vector_index = VectorStoreIndex(
    objects=objects,
)
query_engine = vector_index.as_query_engine(similarity_top_k=1, verbose=True)

### 7. クエリのテスト

In [ ]:
response = query_engine.query("トロントで有名なものはなんですか？")
display(HTML(f'<p style="font-size:20px">{response.response}</p>'))

In [ ]:
response = query_engine.query("ヒューストンは誰がいつ設立しましたか?")
display(HTML(f'<p style="font-size:20px">{response.response}</p>'))

In [ ]:
response = query_engine.query("ボストンのスポーツチームについてまとめてください。")
display(HTML(f'<p style="font-size:20px">{response.response}</p>'))

In [ ]:
response = query_engine.query("シカゴの良い面をすべてまとめてください。")
display(HTML(f'<p style="font-size:20px">{response.response}</p>'))